# Part 1 - a simple keras model for the IMDB dataset

We'll use the IMDB movie review classification dataset built into Keras as the problem. First, let's run that dataset through a simple model. (Code based on the Chollet example) 

In [37]:
import numpy as np
from keras.datasets import imdb
from keras import models
from keras import layers
from keras import optimizers
from keras import regularizers

% matplotlib inline

In [21]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

Let's decode one of these reviews back into English

In [22]:
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join(
    [reverse_word_index.get(i - 3, '?') for i in train_data[0]])

In [23]:
decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [24]:
train_data.shape

(25000,)

Let's prepare the data by padding the lists and turning them into a tensor.

In [25]:
import numpy as np
def vectorize_sequences (sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [26]:
x_train = vectorize_sequences (train_data)
x_test = vectorize_sequences (test_data)

In [27]:
y_train = np.asarray(train_labels).astype('float32')#.reshape(y_train.shape[0], 1)
y_test = np.asarray(test_labels).astype('float32')#.reshape(y_train.shape[0], 1)

x_train now has dimensions (m, n) where m is number of training examples and n is number of features (10,000 possible word features, in this case)

In [28]:
print ("x_train.shape = ", x_train.shape)
print ("x_test.shape = ", x_test.shape)
print ("y_train.shape = ", y_train.shape)
print ("y_test.shape = ", y_test.shape)

x_train.shape =  (25000, 10000)
x_test.shape =  (25000, 10000)
y_train.shape =  (25000,)
y_test.shape =  (25000,)


As we're going to be doing some quite intensive fitting of hyperparameters based on accuracy of the test set, we need a third dev set. Since the test set is generously sized, let's just split that in two.

In [29]:
x_dev = x_test[0: 10000,:]
y_dev = y_test[0: 10000]

x_test = x_test[10000:x_test.shape[0], :]
y_test = y_test[10000:y_test.shape[0]]

In [30]:
print ("x_train.shape = ", x_train.shape)
print ("x_dev.shape = ", x_dev.shape)
print ("x_test.shape = ", x_test.shape)

print ("y_train.shape = ", y_train.shape)
print ("y_dev.shape = ", y_dev.shape)
print ("y_test.shape = ", y_test.shape)

x_train.shape =  (25000, 10000)
x_dev.shape =  (10000, 10000)
x_test.shape =  (15000, 10000)
y_train.shape =  (25000,)
y_dev.shape =  (10000,)
y_test.shape =  (15000,)


# Initialize the network

In [115]:
def build_model(lambd = 0.001, learning_rate = 0.001, dropout_rate = 0.20):
    
    model = models.Sequential()
    model.add (layers.Dense(16, 
                            kernel_regularizer = regularizers.l2(lambd),
                            activation='relu', input_shape = (10000,)))
    model.add (layers.Dropout(float(dropout_rate))) # dropout_rate gets passed in as (1,) array
                                                    # 
    model.add (layers.Dense(16, 
                            kernel_regularizer = regularizers.l2(lambd),
                            activation='relu'))
    model.add (layers.Dropout(float(dropout_rate)))
    model.add (layers.Dense(1, activation='sigmoid'))
    model.compile (optimizer=optimizers.RMSprop(lr = learning_rate), 
               loss = 'binary_crossentropy',
               metrics = ['acc'])
    
    return (model)

In [112]:
import matplotlib.pyplot as plt
acc = history_dict['acc']
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot (epochs, loss_values, 'bo', label = 'Training loss')
plt.plot (epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show

NameError: name 'history_dict' is not defined

In [88]:
plt.clf()

acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot (epochs, acc_values, 'bo', label = 'Training acc')
plt.plot (epochs, val_acc_values, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show

NameError: name 'history_dict' is not defined

<Figure size 432x288 with 0 Axes>

# Hyperparameter Search Algorithm

In [89]:
print ("training accuracy", history.history['acc'][-1])
print ("test accuracy", history.history['val_acc'][-1])
print ("training loss", history.history['loss'][-1])
print ("test loss", history.history['val_loss'][-1])

NameError: name 'history' is not defined

To start with, let's build an algorithm that trains a model for 10 epochs and varies a hyperparameter randomly between two start/end points. Let's just use the learning rate for now. We'll use a log uniform random distribution. We can do this quite easily just by rerunning the `model.compile` step.

In [116]:
from numpy import random

def search_learning_rate(model, 
                         num_models = 10): # how many different values of the lr to try
                                
        
    
# search random values on a log scale to get learning rates
    lr_exponent = (-3 * random.rand(num_models,1)) - 1 # exponent - search between 10^-4 and 10^-1
    learning_rates = 10 ** lr_exponent # lr is a num_iterations x 1 vector
        
    lambd_exponent = (-4 * random.rand(num_models,1)) - 1
    lambdas = 10 ** lambd_exponent
    
    dropout_rand = random.rand(num_models,1)
    dropout_rates = np.round (0.5 * dropout_rand, 2) 
    print (dropout_rates[:10])
    
    # initialize vectors for validation loss
    val_loss = np.zeros ((num_models, 1))
    val_acc = np.zeros ((num_models, 1))
    
    for i in range (num_models):
        
        model = build_model (lambd = lambdas[i], 
                             learning_rate = learning_rates[i], 
                             dropout_rate = dropout_rates[i])
        
        history = model.fit(x_train, y_train, verbose = 0, epochs = 10, batch_size=512, validation_data=(x_dev, y_dev))
        val_loss[i] = history.history['val_loss'][-1]
        val_acc[i] = history.history['val_acc'][-1]
        print (i, "of", num_models, "lr:", learning_rates[i], "lambda:", lambdas[i], "drop %:", dropout_rates[i], "dev loss:", val_loss[i], "dev acc:", val_acc[i])
    return (learning_rates, lambdas, dropout_rates, val_loss, val_acc)

In [117]:
num_models = 100

results_lr, results_lambd, results_val_loss, results_val_acc = search_learning_rate(model, num_models)

print ("learning rates : validation loss : validation accuracy")
for i in range (num_models):
    print (results_lr[i], ":", results_val_loss[i], ":", results_val_acc[i])

Dropout rate:  [0.2]
Dopout rate type: <class 'numpy.ndarray'>
Dopout rate shape: (1,)
Dopout rate float type: <class 'float'>
0 of 100 lr: [0.06987086] lambda: [0.01184456] drop %: [0.2] dev loss: [2.99373146] dev acc: [0.5354]
Dropout rate:  [0.2]
Dopout rate type: <class 'numpy.ndarray'>
Dopout rate shape: (1,)
Dopout rate float type: <class 'float'>


KeyboardInterrupt: 

In [135]:
iterations = 10                                 # how many different values of the lr to try

# search random values on a log scale to get learning rates
exponent = (-3 * random.rand(iterations,1)) - 1 # exponent - search between 10^-4 and 10^-1
lr = 10 ** exponent 

In [139]:
print (val_loss)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
